<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_sept/blob/main/4_pnl/clase_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np

# Ejemplo de datos categóricos
data = {'Categoria': ['Rojo', 'Azul', 'Verde', 'Rojo', 'Azul']}
df = pd.DataFrame(data)

print("DataFrame original:")
print(df)
print("-" * 30)

# Usando pandas get_dummies
df_dummies = pd.get_dummies(df, columns=['Categoria'])
print("Usando pandas get_dummies:")
print(df_dummies)
print("-" * 30)

# Usando Scikit-learn OneHotEncoder
# Necesita un array numpy 2D
data_np = df[['Categoria']].values
encoder_sk = OneHotEncoder(sparse_output=False)
data_encoded_sk = encoder_sk.fit_transform(data_np)
print("Usando Scikit-learn OneHotEncoder:")
print(data_encoded_sk)
print("-" * 30)

# Usando Keras to_categorical
# Necesita etiquetas numéricas. Primero convertimos las categorías a números.
# Usamos factorize para esto
codes, uniques = pd.factorize(df['Categoria'])
data_encoded_keras = to_categorical(codes)
print("Usando Keras to_categorical:")
print(data_encoded_keras)
print("-" * 30)

print("Diferencias clave:")
print("- `pd.get_dummies`: Funciona directamente con DataFrames, crea nuevas columnas con los nombres de las categorías.")
print("- `sklearn.preprocessing.OneHotEncoder`: Trabaja con arrays NumPy (requiere reshape si es necesario), devuelve un array NumPy. Requiere `fit` y `transform`.")
print("- `tf.keras.utils.to_categorical`: Trabaja con etiquetas numéricas (enteros), devuelve un array NumPy. Útil para la salida de modelos de clasificación en Keras.")

DataFrame original:
  Categoria
0      Rojo
1      Azul
2     Verde
3      Rojo
4      Azul
------------------------------
Usando pandas get_dummies:
   Categoria_Azul  Categoria_Rojo  Categoria_Verde
0           False            True            False
1            True           False            False
2           False           False             True
3           False            True            False
4            True           False            False
------------------------------
Usando Scikit-learn OneHotEncoder:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
------------------------------
Usando Keras to_categorical:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
------------------------------
Diferencias clave:
- `pd.get_dummies`: Funciona directamente con DataFrames, crea nuevas columnas con los nombres de las categorías.
- `sklearn.preprocessing.OneHotEncoder`: Trabaja con arrays NumPy (requiere reshape si es necesario), devuelve un array NumPy. Requi

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import numpy as np

# Ejemplo de texto
sentences = [
    'El perro corre en el parque',
    'El gato duerme en la casa',
    'El perro juega con la pelota',
    'El gato caza un ratón'
]

# Tokenización
tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)

print("Índice de palabras:")
print(word_index)
print("-" * 30)

print("Secuencias de texto:")
print(sequences)
print("-" * 30)

# Padding (para que todas las secuencias tengan la misma longitud)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

print("Secuencias con padding:")
print(padded_sequences)
print("-" * 30)

# Crear un modelo simple con una capa de Embedding
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=8, input_length=max_length),
    Flatten(),
    Dense(1, activation='sigmoid') # Capa de ejemplo, puede ser cualquier otra
])

# Build the model to initialize weights
model.build(input_shape=(None, max_length))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Para ver los embeddings, puedes acceder a los pesos de la capa de Embedding
# Ten en cuenta que estos embeddings son aleatorios al principio y se aprenderían durante el entrenamiento
embedding_layer = model.layers[0]
embeddings = embedding_layer.get_weights()[0]

print("Forma de la matriz de embeddings (vocabulario_size, embedding_dim):")
print(embeddings.shape)
print("-" * 30)

# Puedes ver el vector de embedding para una palabra específica (ej. 'perro')
word = 'perro'
if word in word_index:
    index = word_index[word]
    word_embedding = embeddings[index]
    print(f"Vector de embedding para '{word}':")
    print(word_embedding)
else:
    print(f"La palabra '{word}' no está en el vocabulario.")

print("-" * 30)

print("Explicación:")
print("- **Tokenización:** Convierte el texto en secuencias de números (IDs de palabras).")
print("- **Padding:** Asegura que todas las secuencias tengan la misma longitud rellenando con ceros.")
print("- **Capa de Embedding:** Representa cada palabra con un vector de números de menor dimensión.")
print("- **output_dim:** Es la dimensión del vector de embedding para cada palabra.")
print("- **input_dim:** Es el tamaño del vocabulario + 1 (para el padding).")
print("- **input_length:** Es la longitud máxima de las secuencias.")
print("Estos embeddings son inicialmente aleatorios y se ajustarían durante el entrenamiento de un modelo en una tarea específica (como clasificación de texto).")

Índice de palabras:
{'<OOV>': 1, 'el': 2, 'perro': 3, 'en': 4, 'gato': 5, 'la': 6, 'corre': 7, 'parque': 8, 'duerme': 9, 'casa': 10, 'juega': 11, 'con': 12, 'pelota': 13, 'caza': 14, 'un': 15, 'ratón': 16}
------------------------------
Secuencias de texto:
[[2, 3, 7, 4, 2, 8], [2, 5, 9, 4, 6, 10], [2, 3, 11, 12, 6, 13], [2, 5, 14, 15, 16]]
------------------------------
Secuencias con padding:
[[ 2  3  7  4  2  8]
 [ 2  5  9  4  6 10]
 [ 2  3 11 12  6 13]
 [ 2  5 14 15 16  0]]
------------------------------


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 6, 8)           │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 185 (740.00 B)

 Trainable params: 185 (740.00 B)

 Non-trainable params: 0 (0.00 B)

Forma de la matriz de embeddings (vocabulario_size, embedding_dim):
(17, 8)
------------------------------
Vector de embedding para 'perro':
[-0.00524797 -0.0449581   0.03110847 -0.02950633  0.00090873  0.00061829
 -0.0278508   0.01563616]
------------------------------
Explicación:
- **Tokenización:** Convierte el texto en secuencias de números (IDs de palabras).
- **Padding:** Asegura que todas las secuencias tengan la misma longitud rellenando con ceros.
- **Capa de Embedding:** Representa cada palabra con un vector de números de menor dimensión.
- **output_dim:** Es la dimensión del vector de embedding para cada palabra.
- **input_dim:** Es el tamaño del vocabulario + 1 (para el padding).
- **input_length:** Es la longitud máxima de las secuencias.
Estos embeddings son inicialmente aleatorios y se ajustarían durante el entrenamiento de un modelo en una tarea específica (como clasificación de texto).


In [4]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

sentences = [
    'El perro corre en el parque',
    'El gato duerme en la casa',
    'El perro juega con la pelota',
    'El gato caza un ratón'
]

# Tokenización para One-Hot Encoding a nivel de palabra
words = [word for sentence in sentences for word in sentence.lower().split()]
unique_words = sorted(list(set(words)))
vocab_size = len(unique_words)

print("Vocabulario para One-Hot Encoding:")
print(unique_words)
print("-" * 30)

# Crear representaciones One-Hot para cada oración
one_hot_encoded_sentences = []
for sentence in sentences:
    sentence_words = sentence.lower().split()
    one_hot_vector = np.zeros(vocab_size)
    for word in sentence_words:
        if word in unique_words:
            index = unique_words.index(word)
            one_hot_vector[index] = 1  # O podrías contar la frecuencia: one_hot_vector[index] += 1
    one_hot_encoded_sentences.append(one_hot_vector)

print("Representación One-Hot de las oraciones:")
for i, sentence in enumerate(sentences):
    print(f"Oración {i+1}: {sentence}")
    print(one_hot_encoded_sentences[i])
    print("-" * 10)

print("\nComparación con Word Embeddings (basado en la celda anterior):")
print("- **One-Hot Encoding:** Crea un vector binario largo donde cada dimensión representa una palabra única del vocabulario. La dimensión del vector es igual al tamaño del vocabulario. No captura relaciones semánticas entre palabras.")
print("- **Word Embeddings:** Representa cada palabra como un vector denso de menor dimensión. La dimensión del vector es un parámetro (output_dim en Keras). Captura relaciones semánticas (palabras similares tienen vectores similares).")

Vocabulario para One-Hot Encoding:
['casa', 'caza', 'con', 'corre', 'duerme', 'el', 'en', 'gato', 'juega', 'la', 'parque', 'pelota', 'perro', 'ratón', 'un']
------------------------------
Representación One-Hot de las oraciones:
Oración 1: El perro corre en el parque
[0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.]
----------
Oración 2: El gato duerme en la casa
[1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.]
----------
Oración 3: El perro juega con la pelota
[0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0.]
----------
Oración 4: El gato caza un ratón
[0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1.]
----------

Comparación con Word Embeddings (basado en la celda anterior):
- **One-Hot Encoding:** Crea un vector binario largo donde cada dimensión representa una palabra única del vocabulario. La dimensión del vector es igual al tamaño del vocabulario. No captura relaciones semánticas entre palabras.
- **Word Embeddings:** Representa cada palabra como un vector denso de menor dimensión. La dimens

In [5]:
# Acceder al word_index y a los embeddings de la celda WMu9ZuXMU5tR
# Es importante que la celda WMu9ZuXMU5tR se haya ejecutado antes para que estas variables existan.

words_to_show = ['pelota', 'parque', 'perro', 'gato']

print("Vectores de embedding para palabras específicas:")
print("-" * 40)

for word in words_to_show:
    if word in word_index:
        index = word_index[word]
        word_embedding = embeddings[index]
        print(f"Vector de embedding para '{word}':")
        print(word_embedding)
        print("-" * 20)
    else:
        print(f"La palabra '{word}' no está en el vocabulario.")
        print("-" * 20)

Vectores de embedding para palabras específicas:
----------------------------------------
Vector de embedding para 'pelota':
[ 0.02356908 -0.02547683  0.03363779  0.02462841  0.02558625 -0.03036195
  0.03894265  0.04249628]
--------------------
Vector de embedding para 'parque':
[ 0.01349691 -0.00854899 -0.03189993 -0.00283885  0.04491376  0.00339184
  0.03399635  0.03922579]
--------------------
Vector de embedding para 'perro':
[-0.00524797 -0.0449581   0.03110847 -0.02950633  0.00090873  0.00061829
 -0.0278508   0.01563616]
--------------------
Vector de embedding para 'gato':
[ 0.03620568 -0.02313721  0.00301746 -0.01685438 -0.03915597  0.03584406
 -0.00405413  0.02971119]
--------------------


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

words_to_compare = ['pelota', 'parque', 'perro', 'gato']
word_vectors = {}

# Obtener los vectores de embedding para las palabras de interés
for word in words_to_compare:
    if word in word_index:
        index = word_index[word]
        word_vectors[word] = embeddings[index].reshape(1, -1) # Reshape para que sea 2D
    else:
        print(f"Advertencia: La palabra '{word}' no está en el vocabulario.")

print("Medidas de similitud del coseno entre conceptos:")
print("-" * 50)

# Calcular y mostrar la similitud del coseno entre pares de palabras
for i in range(len(words_to_compare)):
    for j in range(i + 1, len(words_to_compare)):
        word1 = words_to_compare[i]
        word2 = words_to_compare[j]

        if word1 in word_vectors and word2 in word_vectors:
            vector1 = word_vectors[word1]
            vector2 = word_vectors[word2]
            similarity = cosine_similarity(vector1, vector2)[0][0]
            print(f"Similitud '{word1}' y '{word2}': {similarity:.4f}")
        else:
            print(f"No se pudo calcular la similitud entre '{word1}' y '{word2}' (una o ambas palabras no encontradas).")

print("-" * 50)

print("Nota: Estos valores de similitud se basan en los embeddings aleatorios iniciales. Después del entrenamiento, los valores reflejarían mejor las relaciones semánticas aprendidas.")

Medidas de similitud del coseno entre conceptos:
--------------------------------------------------
Similitud 'pelota' y 'parque': 0.4999
Similitud 'pelota' y 'perro': 0.1492
Similitud 'pelota' y 'gato': 0.0212
Similitud 'parque' y 'perro': -0.1631
Similitud 'parque' y 'gato': 0.0048
Similitud 'perro' y 'gato': 0.3740
--------------------------------------------------
Nota: Estos valores de similitud se basan en los embeddings aleatorios iniciales. Después del entrenamiento, los valores reflejarían mejor las relaciones semánticas aprendidas.
